In [19]:
import os
os.environ['PATH'] = "D:/108/openslide-win64-20171122/openslide-win64-20171122/bin" + ";" + os.environ['PATH']

In [20]:
# windows記得先載visual studio c++
import cv2
import json
# import openslide
import matplotlib.pyplot as plt
import large_image
import numpy as np
import skimage.io
from skimage import io
# import large_image_source_openslide
from PIL import Image
# from openslide.deepzoom import DeepZoomGenerator
import matplotlib.image as mpimg
from tqdm import tqdm_notebook
import math
import pandas as pd
import MTM  # multi template matching
from MTM import matchTemplates, drawBoxesOnRGB
from shapely.geometry import Polygon
from scipy import stats
import glob
from pandas import json_normalize
from tqdm import tqdm

In [21]:
Image.MAX_IMAGE_PIXELS = 10000000000

In [22]:
all_df = pd.DataFrame(glob.glob(r'./MTM2023/*'),columns=['img_name'])
all_df['img_name'] = all_df['img_name'].apply(lambda x : x.replace("\\","/"))
all_path = pd.DataFrame()
for i in all_df['img_name']:
    for j in os.listdir(i):
        path = {"path": i+"/"+j}
        all_path = pd.concat([all_path,pd.DataFrame([path])],ignore_index=True)
all_path = all_path.reset_index(drop=True)
all_path = all_path.rename({0:'path'},axis=1)
all_path = all_path.sort_values(by=["path"]).reset_index(drop=True)

BFC_path = all_path[all_path["path"].str.contains("PTC")]

In [24]:
BFC_path = BFC_path[1:]
BFC_path = BFC_path.reset_index(drop=True)
BFC_path['name'] = BFC_path['path'].apply(lambda x : x.split("/")[3])

In [25]:
mask_df = pd.DataFrame(glob.glob(r'./PTC/all_0602/*'),columns=['img_name'])
mask_df['img_name'] = mask_df['img_name'].apply(lambda x : x.replace("\\","/"))
mask_df = mask_df.sort_values(by=["img_name"]).reset_index(drop=True)
mask_df['name'] = mask_df['img_name'].apply(lambda x : x.split("/")[3])

In [26]:
img_path = pd.merge(BFC_path, mask_df, on='name')

In [27]:
img_path

,path,name,img_name
0,./MTM2023/PTC_18_014-C/PTC_18_014-C_15.png,PTC_18_014-C_15.png,./PTC/all_0602/PTC_18_014-C_15.png
1,./MTM2023/PTC_18_014-C/PTC_18_014-C_19.png,PTC_18_014-C_19.png,./PTC/all_0602/PTC_18_014-C_19.png
2,./MTM2023/PTC_18_014-C/PTC_18_014-C_20.png,PTC_18_014-C_20.png,./PTC/all_0602/PTC_18_014-C_20.png
3,./MTM2023/PTC_18_014-C/PTC_18_014-C_21.png,PTC_18_014-C_21.png,./PTC/all_0602/PTC_18_014-C_21.png
4,./MTM2023/PTC_18_014-C/PTC_18_014-C_28.png,PTC_18_014-C_28.png,./PTC/all_0602/PTC_18_014-C_28.png
...,...,...,...
2873,./MTM2023/PTC_19_120-C/PTC_19_120-C_90.png,PTC_19_120-C_90.png,./PTC/all_0602/PTC_19_120-C_90.png
2874,./MTM2023/PTC_19_120-C/PTC_19_120-C_93.png,PTC_19_120-C_93.png,./PTC/all_0602/PTC_19_120-C_93.png
2875,./MTM2023/PTC_19_120-C/PTC_19_120-C_96.png,PTC_19_120-C_96.png,./PTC/all_0602/PTC_19_120-C_96.png
2876,./MTM2023/PTC_19_120-C/PTC_19_120-C_97.png,PTC_19_120-C_97.png,./PTC/all_0602/PTC_19_120-C_97.png


In [13]:
img_path['path'][0].split('/')[3]

['.', 'MTM2023', 'BFC_20_001', 'BFC_20_001_0.png']

'./BFC/origin/BFC_20_001_0.png'

In [18]:
for i in img_path["path"]:
    img = cv2.imread(i)
    cv2.imwrite(f'./BFC/origin/{i.split("/")[3]}', img)